In [55]:
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import requests
import re

In [2]:
# prep

og_html = requests.get('https://religion.byu.edu/directory')
s = bs(og_html.text)

In [10]:
# stuff that works

profs = s.find_all('h3', class_="PromoVerticalImage-title promo-title")

names = [i.find('a').text for i in profs]

links = [i.find('a')['href'] for i in profs]

titles = [None] * len(profs)
for i in range(len(profs)):
    try:
        titles[i] = profs[i].parent.find(class_="PromoVerticalImage-jobTitle").text
    except AttributeError:
        titles[i] = None
        
areas = [None] * len(profs)
for i in range(len(profs)):
    try:
        areas[i] = profs[i].parent.find(class_="PromoVerticalImage-groups").text
    except AttributeError:
        areas[i] = None
areas = [i.split(', ') for i in areas]

slc = [False] * len(profs)
for i in range(len(profs)):
    if 'Salt Lake Center' in areas[i]:
        slc[i] = True
        areas[i].remove('Salt Lake Center')

In [105]:
# stuff I'm working on
# This takes about 20 seconds to run
# It successfully gets all the offices, but I need to clean it up--turn 'Joseph Smith Building' into JSB and split
# into two columns, building and room number

offices = [None] * len(profs)
for i in range(len(profs)):
    html = requests.get(links[i])
    s = bs(html.text)
    t = s.find(class_="RichTextModule-items RichTextBody").text
    offices[i] = t.strip(' \n').split(' \n')[0]